In [1]:
import pandas as pd

In [23]:
df = pd.read_csv("EURUSD_Daily.csv")
df

,Gmt time,Open,High,Low,Close,Volume
0,01.01.2020 00:00:00.000,1.12120,1.12218,1.12106,1.12188,2621.3201
1,02.01.2020 00:00:00.000,1.12188,1.12245,1.11635,1.11708,487219.1250
2,03.01.2020 00:00:00.000,1.11707,1.11798,1.11249,1.11548,177406.7969
3,05.01.2020 00:00:00.000,1.11656,1.11677,1.11589,1.11604,4398.1001
4,06.01.2020 00:00:00.000,1.11603,1.12055,1.11571,1.11954,139965.2813
...,...,...,...,...,...,...
1268,22.01.2024 00:00:00.000,1.08920,1.09094,1.08768,1.08772,233830.1400
1269,23.01.2024 00:00:00.000,1.08776,1.09162,1.08215,1.08572,278337.1821
1270,24.01.2024 00:00:00.000,1.08573,1.09323,1.08514,1.08810,357244.5010
1271,25.01.2024 00:00:00.000,1.08811,1.09018,1.08216,1.08454,364079.3330


# Fixed horizon labeling

In [4]:
def label_threshold(df, threshold, bars):
    lower_threshold = df['Close'] - threshold
    upper_threshold = df['Close'] + threshold
    df['Label'] = 0
    for i in range(len(df)-bars):
        if any(df.loc[i:i+bars, 'Close'] > upper_threshold[i]):
            df.loc[i, 'Label'] = 1
        elif any(df.loc[i:i+bars, 'Close'] < lower_threshold[i]):
            df.loc[i, 'Label'] = -1
    return df

label_threshold(df, 0.001, 10)

,Gmt time,Open,High,Low,Close,Volume,Label
0,01.01.2020 00:00:00.000,1.12120,1.12218,1.12106,1.12188,2621.3201,-1
1,02.01.2020 00:00:00.000,1.12188,1.12245,1.11635,1.11708,487219.1250,1
2,03.01.2020 00:00:00.000,1.11707,1.11798,1.11249,1.11548,177406.7969,1
3,05.01.2020 00:00:00.000,1.11656,1.11677,1.11589,1.11604,4398.1001,1
4,06.01.2020 00:00:00.000,1.11603,1.12055,1.11571,1.11954,139965.2813,-1
...,...,...,...,...,...,...,...
1268,22.01.2024 00:00:00.000,1.08920,1.09094,1.08768,1.08772,233830.1400,0
1269,23.01.2024 00:00:00.000,1.08776,1.09162,1.08215,1.08572,278337.1821,0
1270,24.01.2024 00:00:00.000,1.08573,1.09323,1.08514,1.08810,357244.5010,0
1271,25.01.2024 00:00:00.000,1.08811,1.09018,1.08216,1.08454,364079.3330,0


# Volatility related labeling

In [24]:
df["Gmt time"]=df["Gmt time"].str.replace(".000","")
df['Gmt time']=pd.to_datetime(df['Gmt time'],format='%d.%m.%Y %H:%M:%S')
df=df[df.High!=df.Low]
df.set_index("Gmt time", inplace=True)

In [25]:
df

,Open,High,Low,Close,Volume
Gmt time,,,,,
2020-01-01,1.12120,1.12218,1.12106,1.12188,2621.3201
2020-01-02,1.12188,1.12245,1.11635,1.11708,487219.1250
2020-01-03,1.11707,1.11798,1.11249,1.11548,177406.7969
2020-01-05,1.11656,1.11677,1.11589,1.11604,4398.1001
2020-01-06,1.11603,1.12055,1.11571,1.11954,139965.2813
...,...,...,...,...,...
2024-01-22,1.08920,1.09094,1.08768,1.08772,233830.1400
2024-01-23,1.08776,1.09162,1.08215,1.08572,278337.1821
2024-01-24,1.08573,1.09323,1.08514,1.08810,357244.5010


In [26]:
def calculate_daily_volatility(close_prices, span=100):
    # Find indices for the previous day
    previous_day_indices = close_prices.index.searchsorted(close_prices.index - pd.Timedelta(days=1))
    previous_day_indices = previous_day_indices[previous_day_indices > 0]

    # Reindex to the previous day and calculate daily returns
    reindexed_close = pd.Series(close_prices.index[previous_day_indices - 1],
                                index=close_prices.index[close_prices.shape[0] - previous_day_indices.shape[0]:])
    daily_returns = close_prices.loc[reindexed_close.index] / close_prices.loc[reindexed_close.values].values - 1

    # Calculate daily volatility using exponentially weighted moving standard deviation
    daily_volatility = daily_returns.ewm(span=span).std()

    return daily_volatility

In [27]:
def volatility_label_threshold(df, bars):
    threshold = calculate_daily_volatility(df.Close)
    lower_threshold = df['Close'] - threshold
    upper_threshold = df['Close'] + threshold
    df = df.reset_index()

    df['Label_D'] = 0
    for i in range(len(df)-bars):
        if any(df.loc[i:i+bars, 'Close'] > upper_threshold[i]):
            df.loc[i, 'Label_D'] = 1
        elif any(df.loc[i:i+bars, 'Close'] < lower_threshold[i]):
            df.loc[i, 'Label_D'] = -1
    return df

In [28]:
df = volatility_label_threshold(df, 5)

In [34]:
import numpy as np
def pointpos(x):
    if x['Label_ATR']==1:
        return x['Low']-1e-3
    elif x['Label_ATR']==-1:
        return x['High']+1e-3
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [35]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
st=100
dfpl = df[st:st+350]
#dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="entry")

fig.show()

# ATR related volatility and labeling

In [31]:
import pandas_ta as ta
def atr_label_threshold(df, bars):
    df['ATR'] = ta.atr(df['High'], df['Low'], df['Close'])

    lower_threshold = df['Close'] - df['ATR']*2
    upper_threshold = df['Close'] + df['ATR']*2

    df = df.reset_index()

    df['Label_ATR'] = 0
    for i in range(len(df)-bars):
        if any(df.loc[i:i+bars, 'Close'] > upper_threshold[i]):
            df.loc[i, 'Label_ATR'] = 1
        elif any(df.loc[i:i+bars, 'Close'] < lower_threshold[i]):
            df.loc[i, 'Label_ATR'] = -1
    return df

In [32]:
df = atr_label_threshold(df, 10)

In [33]:
df

,index,Gmt time,Open,High,Low,Close,Volume,Label_D,pointpos,ATR,Label_ATR
0,0,2020-01-01,1.12120,1.12218,1.12106,1.12188,2621.3201,0,NaN,NaN,0
1,1,2020-01-02,1.12188,1.12245,1.11635,1.11708,487219.1250,0,NaN,NaN,0
2,2,2020-01-03,1.11707,1.11798,1.11249,1.11548,177406.7969,0,NaN,NaN,0
3,3,2020-01-05,1.11656,1.11677,1.11589,1.11604,4398.1001,-1,1.11777,NaN,0
4,4,2020-01-06,1.11603,1.12055,1.11571,1.11954,139965.2813,-1,1.12155,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
1268,1268,2024-01-22,1.08920,1.09094,1.08768,1.08772,233830.1400,0,NaN,0.004945,0
1269,1269,2024-01-23,1.08776,1.09162,1.08215,1.08572,278337.1821,0,NaN,0.005268,0
1270,1270,2024-01-24,1.08573,1.09323,1.08514,1.08810,357244.5010,0,NaN,0.005470,0
1271,1271,2024-01-25,1.08811,1.09018,1.08216,1.08454,364079.3330,0,NaN,0.005652,0
